In [1]:
from keras.layers import Input
from keras.optimizers import SGD,Adam,RMSprop
from keras.layers import Bidirectional
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer()

Using TensorFlow backend.


In [2]:
corpus=[]
for line in open('data2.txt','r').readlines():
    corpus.append(line.strip())

tokenizer.fit_on_texts(corpus)
sequences=tokenizer.texts_to_sequences(corpus)

data_oh=pad_sequences(sequences=sequences,maxlen=50,padding='post')   #最大长度为50，以post的方式补全

#导入数据，对案件类型分类
filepath='C:/Users/zhangsheng/Desktop/DSjjcl/version2/new_all_df.csv'    #原始数据 
all_df=pd.read_csv(filepath,encoding='gb18030')

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labEn=LabelEncoder()

all_df['案件类型']=labEn.fit_transform(all_df['案件类型'])
new_df=all_df[['案件类型']]

all_labels=new_df.values

In [ ]:
all_labels[:5]

In [ ]:
corpus

In [ ]:
data_oh

In [3]:
word_index=tokenizer.word_index
Embedding_dim=200
max_length=50

In [4]:
from keras.utils import to_categorical

labels=to_categorical(all_labels)

In [5]:
labels

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [5]:
#分割数据  8:2的比例  随机
msk=np.random.rand(len(new_df))<0.8
train=new_df[msk]
test=new_df[~msk]

index=train.index
x_train=data_oh[index][0:40000]
x_val=data_oh[index][40000:]

# y_train=train.values[0:40000]
# y_val=train.values[40000:]
y_train=labels[index][0:40000]
y_val=labels[index][40000:]

test_index = test.index
x_test = data_oh[test_index]
y_test=labels[test_index]

In [ ]:
y_train[:2]

In [6]:
from keras.layers import Dense,Input,Flatten,Dropout,LSTM,Embedding
from keras.models import Sequential

In [7]:
from keras_self_attention import SeqSelfAttention

In [8]:
model = Sequential()
model.add(Embedding(input_dim=len(word_index)+1,
                   output_dim=Embedding_dim, input_length=max_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
#model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(Dense(3, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 200)           3023000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 3,191,835
Trainable params: 3,191,835
Non-trainable params: 0
_________________________________________________________________


In [9]:
from keras.callbacks import EarlyStopping

In [19]:
early_stopping=EarlyStopping(monitor='acc',patience=20)
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(),
             metrics=['accuracy'])
model.fit(x_train,y_train,validation_data=[x_val,y_val],epochs=50,batch_size=128)

Train on 40000 samples, validate on 14828 samples
Epoch 1/50
40000/40000 [==============================] - 59s 1ms/step - loss: 1.0572 - acc: 0.4629 - val_loss: 1.0556 - val_acc: 0.4850
Epoch 2/50
40000/40000 [==============================] - 59s 1ms/step - loss: 1.0561 - acc: 0.4638 - val_loss: 1.0520 - val_acc: 0.4850
Epoch 3/50
40000/40000 [==============================] - 58s 1ms/step - loss: 1.0563 - acc: 0.4638 - val_loss: 1.0562 - val_acc: 0.4850
Epoch 4/50
40000/40000 [==============================] - 59s 1ms/step - loss: 1.0559 - acc: 0.4638 - val_loss: 1.0529 - val_acc: 0.4850
Epoch 5/50
40000/40000 [==============================] - 58s 1ms/step - loss: 1.0560 - acc: 0.4638 - val_loss: 1.0538 - val_acc: 0.4850
Epoch 6/50
40000/40000 [==============================] - 61s 2ms/step - loss: 1.0559 - acc: 0.4638 - val_loss: 1.0563 - val_acc: 0.4850
Epoch 7/50
40000/40000 [==============================] - 59s 1ms/step - loss: 1.0559 - acc: 0.4638 - val_loss: 1.0548 - val_acc

In [8]:
###载入预训练好的word2vec模型，并使用该模型替代Embedding层
import gensim
from gensim.models import Word2Vec

d:\python3.6\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
w2v_model=Word2Vec.load('word2vec.model')
embedding_matrix=np.zeros((len(word_index)+1,Embedding_dim))
for word,i in word_index.items():
    if word in w2v_model:
        embedding_matrix[i]=np.asarray(w2v_model[word],
                                      dtype='float32')
embedding_layer=Embedding(len(word_index)+1,
                         Embedding_dim,   #输出维度应该=预训练的size大小
                         weights=[embedding_matrix],
                         input_length=max_length,
                         trainable=False)

d:\python3.6\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
d:\python3.6\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [10]:
###LSTM+word2vec
model_=Sequential()
model_.add(embedding_layer)
model_.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model_.add(Dropout(0.2))
model_.add(Dense(3,activation='softmax'))
model_.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 200)           3023000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 3,191,835
Trainable params: 168,835
Non-trainable params: 3,023,000
_________________________________________________________________


In [10]:
model_.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(),
             metrics=['accuracy'])
model_.fit(x_train,y_train,validation_data=[x_val,y_val],epochs=10,batch_size=128)

Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 15040 samples
Epoch 1/10
40000/40000 [==============================] - 87s 2ms/step - loss: 0.5250 - acc: 0.7389 - val_loss: 0.3735 - val_acc: 0.8975
Epoch 2/10
40000/40000 [==============================] - 78s 2ms/step - loss: 0.3320 - acc: 0.9014 - val_loss: 0.3028 - val_acc: 0.9189
Epoch 3/10
40000/40000 [==============================] - 78s 2ms/step - loss: 0.3252 - acc: 0.9049 - val_loss: 0.2897 - val_acc: 0.9178
Epoch 4/10
40000/40000 [==============================] - 80s 2ms/step - loss: 0.2850 - acc: 0.9141 - val_loss: 0.2525 - val_acc: 0.9257
Epoch 5/10
40000/40000 [==============================] - 81s 2ms/step - loss: 0.2612 - acc: 0.9139 - val_loss: 0.2121 - val_acc: 0.9342
Epoch 6/10
40000/40000 [==============================] - 79s 2ms/step - loss: 0.2150 - acc: 0.9288 - val_loss: 0.1624 - val_acc: 0.9485
Epoch 7/10
40000/40000 [==============================] - 81s 2ms/step - loss: 0

In [ ]:
'''
结论：
    1。word2vec训练时间每个epoch节省了20s左右
    2.准确率稍有提升
'''

In [12]:
prediction=model_.predict(x_test)
print(metrics.classification_report(np.argmax(prediction,axis=1),np.argmax(y_test,axis=1),digits=4))

              precision    recall  f1-score   support

           0     0.9769    0.9809    0.9789      6339
           1     0.8986    0.9282    0.9132      2951
           2     0.9572    0.9301    0.9435      4350

   micro avg     0.9533    0.9533    0.9533     13640
   macro avg     0.9442    0.9464    0.9452     13640
weighted avg     0.9537    0.9533    0.9534     13640



In [10]:
####BiLSTM

Bi_model=Sequential()
Bi_model.add(Embedding(input_dim=len(word_index)+1,output_dim=Embedding_dim,input_length=max_length))
Bi_model.add(Bidirectional(LSTM(128,dropout=0.2,recurrent_dropout=0.2)))
# Bi_model.add(SeqSelfAttention(attention_activation='sigmoid'))
Bi_model.add(Dropout(0.2))
Bi_model.add(Dense(3,activation='softmax'))
Bi_model.summary()
Bi_model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=["accuracy"])
Bi_model.fit(x_train,y_train,validation_data=[x_val,y_val],epochs=10,batch_size=128)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 200)           3023000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               336896    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 3,360,667
Trainable params: 3,360,667
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, valid

In [13]:
from sklearn import metrics

In [15]:
prediction=Bi_model.predict(x_test)
print(metrics.classification_report(np.argmax(prediction,axis=1),np.argmax(y_test,axis=1),digits=4))

              precision    recall  f1-score   support

           0     0.9858    0.9857    0.9857      6488
           1     0.9202    0.9400    0.9300      3068
           2     0.9582    0.9431    0.9506      4112

   micro avg     0.9626    0.9626    0.9626     13668
   macro avg     0.9548    0.9563    0.9555     13668
weighted avg     0.9628    0.9626    0.9627     13668



In [11]:
###Bi-LSTM +Word2Vec
Bi_w2v_model=Sequential()
Bi_w2v_model.add(embedding_layer)
Bi_w2v_model.add(Bidirectional(LSTM(128,dropout=0.5,recurrent_dropout=0.5)))
Bi_w2v_model.add(Dense(3,activation='softmax'))
Bi_w2v_model.summary()
Bi_w2v_model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
Bi_w2v_model.fit(x_train,y_train,validation_data=[x_val,y_val],epochs=10,batch_size=128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 200)           3023000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               336896    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
Total params: 3,360,667
Trainable params: 337,667
Non-trainable params: 3,023,000
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 14865 samples
Epoch 1/10
40000/40000 [==============================] - 95s 2ms/step - loss: 0.2589 - acc: 0.9076 - val_loss: 0.1704 - val_acc: 0.9452
Epoch 2/10
40000/40000 [==============================] - 93s 2ms/step - loss: 0.1896 - acc: 0.9330 - val_loss: 0.1500 - val_acc: 0.9502
Epoch 3/10
40000/4

In [14]:
prediction=Bi_w2v_model.predict(x_test)
print(metrics.classification_report(np.argmax(prediction,axis=1),np.argmax(y_test,axis=1),digits=4))

              precision    recall  f1-score   support

           0     0.9810    0.9756    0.9783      6441
           1     0.9078    0.9306    0.9190      3226
           2     0.9478    0.9373    0.9425      4148

   micro avg     0.9536    0.9536    0.9536     13815
   macro avg     0.9455    0.9478    0.9466     13815
weighted avg     0.9539    0.9536    0.9537     13815



In [9]:
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

In [10]:
####  CNN+Bi-lstm
c_b_model=Sequential()
c_b_model.add(embedding_layer)
c_b_model.add(Dropout(0.2))
c_b_model.add(Conv1D(64,3,padding='valid',activation='relu',strides=1))
c_b_model.add(MaxPooling1D(3))
c_b_model.add(Bidirectional(LSTM(128,dropout=0.5,recurrent_dropout=0.5)))
c_b_model.add(Dense(3))
c_b_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 200)           3023000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 200)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 48, 64)            38464     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 64)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               197632    
_________________________________________________________________
dense_1 (Dense)      

In [14]:
from keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='acc',patience=20)

In [ ]:
c_b_model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
c_b_model.fit(x_train,y_train,validation_data=[x_val,y_val],epochs=200
              ,batch_size=128)

Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 14792 samples
Epoch 1/200
40000/40000 [==============================] - 71s 2ms/step - loss: 4.5656 - acc: 0.4030 - val_loss: 4.2878 - val_acc: 0.3582
Epoch 2/200
40000/40000 [==============================] - 63s 2ms/step - loss: 6.4262 - acc: 0.4286 - val_loss: 6.8484 - val_acc: 0.3009
Epoch 3/200
40000/40000 [==============================] - 63s 2ms/step - loss: 6.6343 - acc: 0.3497 - val_loss: 4.2823 - val_acc: 0.2500
Epoch 4/200
40000/40000 [==============================] - 66s 2ms/step - loss: 9.4646 - acc: 0.2243 - val_loss: 12.5212 - val_acc: 0.2500
Epoch 5/200
40000/40000 [==============================] - 63s 2ms/step - loss: 10.2836 - acc: 0.2551 - val_loss: 5.8525 - val_acc: 0.2504
Epoch 6/200
40000/40000 [==============================] - 63s 2ms/step - loss: nan - acc: 0.3473 - val_loss: nan - val_acc: 0.4843
Epoch 7/200
40000/40000 [==============================] - 63s 2ms/step - loss